In [ ]:
!pip install torchtext==0.17.2



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 20.3 MB/s  0:00:20
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 45.0 MB/s  0:00:08
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 166.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 170.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 33.4 MB/s  0:00:14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 68.4 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 66.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 67.7 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 72.3 MB/s  0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 64.8 MB/s  0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 63.9 MB/s  0:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from collections import Counter


In [ ]:
def predict_sentiment(model, sentence):
    model.eval()
    with torch.no_grad():
        # Tokenize the input sentence and convert to tensor
        tokens = torch.tensor(text_pipeline(sentence), dtype=torch.long).unsqueeze(0).to(device)
        output = model(tokens)
        prediction = torch.argmax(output, dim=1).item()
        return "Positive" if prediction == 1 else "Negative"


In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from torchtext.vocab import build_vocab_from_iterator
import time

# Load IMDb dataset
dataset = load_dataset("imdb")
train_data = dataset["train"]
test_data = dataset["test"]

# Preprocessing
tokenizer = lambda x: x.lower().split()
def yield_tokens(data):
    for item in data:
        yield tokenizer(item["text"])

vocab = build_vocab_from_iterator(yield_tokens(train_data), specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: 1 if x == "pos" or x == 1 else 0

# Collate function
def collate_batch(batch):
    label_list, text_list = [], []
    for sample in batch:
        label = sample["label"]
        text = torch.tensor(text_pipeline(sample["text"]), dtype=torch.long)
        label_list.append(torch.tensor(label_pipeline(label), dtype=torch.long))
        text_list.append(text)
    text_list = pad_sequence(text_list, batch_first=True, padding_value=vocab["<pad>"])
    label_list = torch.tensor(label_list, dtype=torch.long)
    return label_list.to(device), text_list.to(device)

# DataLoaders
batch_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

# RNN model
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab["<pad>"])
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        return self.fc(hidden.squeeze(0))

model = RNN(len(vocab), embed_dim=64, hidden_dim=128, output_dim=2).to(device)

# Training setup
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

def train_epoch():
    model.train()
    total_acc, total_loss = 0, 0
    for labels, texts in train_loader:
        preds = model(texts)
        loss = criterion(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_acc += (preds.argmax(1) == labels).sum().item()
    return total_loss / len(train_loader), total_acc / len(train_data)

# Train 1 epoch
start = time.time()
loss, acc = train_epoch()
print(f"Train Loss: {loss:.4f} | Accuracy: {acc:.4f} | Time: {time.time() - start:.2f}s")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/lib/python3.11/importlib/__init__.py:126: UserWarning: A NumPy version >=1.25.2 and <2.6.0 is required for this version of SciPy (detected version 1.24.3)
  return _bootstrap._gcd_import(name[level:], package, level)


Train Loss: 0.6944 | Accuracy: 0.4967 | Time: 7.49s


In [ ]:
print(predict_sentiment(model, "I did not love the movie. It was really well-made and emotional."))
print(predict_sentiment(model, "This movie was terrible. Total waste of time."))
print(predict_sentiment(model, "It was okay, not bad but not great either."))



Negative
Negative
Positive


In [ ]:
your_input = input("Enter a movie review: ")
print("Predicted Sentiment:", predict_sentiment(model, your_input))


Enter a movie review: good
Predicted Sentiment: Positive


In [ ]:
your_input = input("Enter a movie review: ")
print("Predicted Sentiment:", predict_sentiment(model, your_input))


Enter a movie review: bad
Predicted Sentiment: Negative


In [ ]:
torch.save(vocab, "vocab.pth")


In [ ]:
torch.save(model.state_dict(), "sentiment_model.pth")


In [ ]:
import torch
torch.save(vocab, "vocab.pth")



In [ ]:
from google.colab import files
files.download("vocab.pth")
files.download("sentiment_model.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("sentiment_model.pth")
files.download("vocab.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>